In [ ]:
import torchvision

from torchvision import transforms

from mlp_mixer.model.mlp_mixer import MLPMixer

In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset1 = torchvision.datasets.MNIST('./data', train=True, download=True,
                                      transform=transform)
dataset2 = torchvision.datasets.MNIST('./data', train=False,
                                      transform=transform)

RemoteDisconnected: Remote end closed connection without response

In [ ]:
model = MLPMixer(image_size=224, patch_size=16)